# Huggingface SageMaker-SDK - BERT Japanese example

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Permissions](#Permissions)
3. [Processing](#Preprocessing)   
    1. [Tokenization](#Tokenization)  
    2. [Uploading data to sagemaker_session_bucket](#Uploading-data-to-sagemaker_session_bucket)  
4. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    3. [Attach to old training job to an estimator ](#Attach-to-old-training-job-to-an-estimator)  
5. [_Coming soon_:Push model to the Hugging Face hub](#Push-model-to-the-Hugging-Face-hub)

# Introduction

このnotebookは[HuggingFaceのSageMaker example](https://github.com/huggingface/notebooks/tree/master/sagemaker)内の以下3つのnotebookの内容を統合し、日本語データで動作する様に変更を加えたものです。    
- 01_getting_started_pytorch
- 05_spot_instances
- 06_sagemaker_metrics


このデモでは、AmazonSageMakerのHuggingFace Estimatorを使用してSageMakerのトレーニングジョブを実行します。    
トレーニングジョブ(`./scripts/train.py`)では`HuggingFace transformers`の`Trainer`クラスを使用してテキストの二値分類を実行します。    
トレーニングジョブの実行にはスポットインスタンスを使用します。

_**NOTE: このデモは、SagemakerNotebookインスタンス、Sagemaker Studio、ローカル環境で実行できます**_

# Development Environment and Permissions

## Installation

このNotebookはSageMakerの`conda_pytorch_p36`カーネルを利用しています。    
日本語処理のため、`transformers`ではなく`transformers[ja]`をインスールします。

_Note: 異なるカーネルを使用する場合はpytorchのインストールも必要になります。_

In [ ]:
!pip install "sagemaker>=2.31.0" "transformers[ja]==4.4.2" "datasets[s3]==1.5.0" --upgrade

## Permissions

ローカル環境でSagemakerを使用する場合はSagemakerに必要な権限を持つIAMロールにアクセスする必要があります。[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください

In [ ]:
import sagemaker


sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# データの準備

Amazon の商品レビューデータセットは[Registry of Open Data on AWS](https://registry.opendata.aws/)で公開されており、 以下からダウンロード可能です。    
このノートブックでは、日本語のデータセットをダウンロードします。

- データセットの概要    
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）    
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil


download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

# Preprocessing

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除し、2クラス分類 (positive が 1, negative が 0)となるようにデータを加工します。
2. 学習データ、テストデータに分けます。

今回利用しないデータは以下の2つです。必要なデータだけ選んで保存します。

- star_rating (評価)とreview_body (レビューのテキストデータ)以外のデータ
- star_rating が 3 のデータ (positive でも negative でもないデータ)

また、評価が1, 2 のデータはラベル 0 (negative) に、評価が4, 5 のデータはラベル 1 (positive) にします。

In [ ]:
import pandas as pd


df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

print(df_pos_neg.shape)
df_pos_neg.head()

In [ ]:
# デモ用にサンプリングしてデータを小さくします
df_pos_neg = df_pos_neg.sample(20000, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_pos_neg, test_size=0.2, random_state=42)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# 後の加工のため、データをDataFrameからHuggingface Datasetsクラスに変換します

import datasets

train = datasets.Dataset.from_pandas(train)
test = datasets.Dataset.from_pandas(test)

In [ ]:
print(train)
print(test)

## Tokenization 

`review_body`は文章テキストのままだとモデルの学習ができないため、テキストを意味のある単位で分割（トークナイズ）した上で機械学習モデルで扱える様に数値に変換します。    
HuggingFaceのTransformersでは、東北大学が公開している[日本語BERT](https://github.com/cl-tohoku/bert-japanese)の学習済みモデル、トークナイザが使用できるため、このデモではこちらを使用します。

In [ ]:
from transformers import BertJapaneseTokenizer
#from transformers import AutoTokenizer


tokenizer_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# download tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained(tokenizer_name)
# download tokenizer
#tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['review_body'], max_length=256, padding='max_length', truncation=True)

試しに一つのテキストでトークナイズと数値変換（エンコード）してみます。    
このデモで使用している`cl-tohoku/bert-base-japanese-whole-word-masking`は32,000個の単語と数値のマッピングがあり、[CLS] = 2, 'ハワイ' = 6166, '##アン' = 1028..., [SEP] = 3などとテキストを数値に変換します。

In [ ]:
print('Original: ', train[0]['review_body'])

print('\nTokenize: ', tokenizer.tokenize(train[0]['review_body']))

print('\nEncode: ', tokenizer.encode(train[0]['review_body']))

この処理をデータ全体に適用します

In [ ]:
# tokenize dataset
train_dataset = train.map(tokenize, batched=True, batch_size=len(train))
test_dataset = test.map(tokenize, batched=True, batch_size=len(test))

In [ ]:
print(train_dataset)
print(test_dataset)

In [ ]:
train_dataset[0]

train_dataset, test_datasetをトレーニングに使用できる様に少し修正します。
- `star_rating`を`labels`に変更します（`Trainer`クラスのデフォルトでは`labels`をターゲットカラム名として認識します）。
- 学習に使用するカラムをpytorchのフォーマットにします。

In [ ]:
# set format for pytorch
train_dataset.rename_column_("star_rating", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

test_dataset.rename_column_("star_rating", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

## Uploading data to `sagemaker_session_bucket`

`datasets`の`FileSystem` [Integration](https://huggingface.co/docs/datasets/filesystems.html)を使用してS3へデータをアップロードします。

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem


s3_prefix = 'samples/datasets/amazon-review'
s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path, fs=s3)

In [ ]:
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'

# 以下のpathにdatasetがuploadされました
print(training_input_path)
print(test_input_path)

# Fine-tuning & starting Sagemaker Training Job

`HuggingFace`のトレーニングジョブを作成するためには`HuggingFace` Estimatorが必要になります。    
Estimatorは、エンドツーエンドのAmazonSageMakerトレーニングおよびデプロイタスクを処理します。 Estimatorで、どのFine-tuningスクリプトを`entry_point`として使用するか、どの`instance_type`を使用するか、どの`hyperparameters`を渡すかなどを定義します。


```python
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    base_job_name='huggingface-sdk-extension',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    role=role,
    hyperparameters={
        'epochs': 1,
        'train_batch_size': 32,
        'model_name':'distilbert-base-uncased'
    }
)
```

SageMakerトレーニングジョブを作成すると、SageMakerは`huggingface`コンテナを実行するために必要なec2インスタンスの起動と管理を行います。    
Fine-tuningスクリプト`train.py`をアップロードし、`sagemaker_session_bucket`からコンテナ内の`/opt/ml/input/data`にデータをダウンロードして、トレーニングジョブを実行します。


```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

`HuggingFace estimator`で定義した`hyperparameters`は、名前付き引数として渡されます。

またSagemakerは、次のようなさまざまな環境変数を通じて、トレーニング環境に関する有用なプロパティを提供しています。

* `SM_MODEL_DIR`：トレーニングジョブがモデルアーティファクトを書き込むパスを表す文字列。トレーニング後、このディレクトリのアーティファクトはモデルホスティングのためにS3にアップロードされます。

* `SM_NUM_GPUS`：ホストで使用可能なGPUの数を表す整数。

* `SM_CHANNEL_XXXX`：指定されたチャネルの入力データを含むディレクトリへのパスを表す文字列。たとえば、HuggingFace estimatorのfit呼び出しで`train`と`test`という名前の2つの入力チャネルを指定すると、環境変数`SM_CHANNEL_TRAIN`と`SM_CHANNEL_TEST`が設定されます。

このトレーニングジョブをローカル環境で実行するには、`instance_type='local'`、GPUの場合は`instance_type='local-gpu'`で定義できます。
_Note：これはSageMaker Studio内では機能しません_

In [ ]:
!pygmentize ./scripts/train.py

## Creating an Estimator and start a training job

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'epochs': 3,
    'train_batch_size': 32,
    'learning_rate' : 5e-5,
    'model_name':'cl-tohoku/bert-base-japanese-whole-word-masking',
    'output_dir':'/opt/ml/checkpoints',
}

# s3 uri where our checkpoints will be uploaded during training
job_name = "using-spot"
checkpoint_s3_uri = f's3://{sess.default_bucket()}/{job_name}/checkpoints'

In [ ]:
print(checkpoint_s3_uri)

トレーニングジョブのメトリック抽出に使用する正規表現ベースの定義を作成します。

In [ ]:
# metric definition to extract the results
metric_definitions=[
    {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}
]

In [ ]:
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    base_job_name=job_name,
    checkpoint_s3_uri=checkpoint_s3_uri,
    use_spot_instances=True,
    max_wait=7200, # This should be equal to or greater than max_run in seconds'
    max_run=3600, # expected max run in seconds
    role=role,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

# データ数: 20,000, 3epochでの実行時間の目安(spotインスタンスの空き具合によって実行時間は変動します)
# Training seconds: 1218
# Billable seconds: 365
# Managed Spot Training savings: 70.0%

## Estimator Parameters

In [ ]:
# container image used for training job
print(f"container image used for training job: \n{huggingface_estimator.image_uri}\n")

# s3 uri where the trained model is located
print(f"s3 uri where the trained model is located: \n{huggingface_estimator.model_data}\n")

# latest training job name for this estimator
print(f"latest training job name for this estimator: \n{huggingface_estimator.latest_training_job.name}\n")

In [ ]:
# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

## Accessing Training Metrics

トレーニングジョブは、メトリックをすぐには出力しません。 たとえば、最初にトレーニングインスタンスをプロビジョニングし、トレーニングイメージをダウンロードし、データをダウンロードする必要があります。 さらに、このデモでは、最初の評価ログは500ステップ後に取得されます（Hugging Face　Trainerクラスのlogging_stepsの[デフォルト](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)。

したがって、**トレーニングを開始してから15〜20分後に以下のセクションを実行してください。そうしないと、まだ利用可能な指標がなく、エラーが返される可能性があります**

`TrainingJobAnalytics` API callで正確なトレーニングジョブ名を指定することで、このコードをコピーして別の場所から実行することもできます（クラウドに接続され、APIの使用が許可されている場合）。

In [ ]:
from sagemaker import TrainingJobAnalytics

# Captured metrics can be accessed as a Pandas dataframe
df = TrainingJobAnalytics(training_job_name=huggingface_estimator.latest_training_job.name).dataframe()
df

In [ ]:
!pip install seaborn

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = [15,5]

In [ ]:
evals = df[df.metric_name.isin(['eval_accuracy','eval_precision'])]
losses = df[df.metric_name.isin(['loss', 'eval_loss'])]

sns.lineplot(
    x='timestamp', 
    y='value', 
    data=evals, 
    hue='metric_name', 
    palette=['blue', 'purple'])

ax2 = plt.twinx()
sns.lineplot(
    x='timestamp', 
    y='value', 
    data=losses, 
    hue='metric_name', 
    palette=['orange', 'red'],
    ax=ax2)

## Download-fine-tuned-model-from-s3

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

## Attach to old training job to an estimator 

Sagemakerでは、古いトレーニングジョブをEstimatorにアタッチして、トレーニングを続けたり、結果を取得したりできます。

In [ ]:
from sagemaker.estimator import Estimator

# job which is going to be attached to the estimator
old_training_job_name=''

In [ ]:
# attach old training job
huggingface_estimator_loaded = Estimator.attach(old_training_job_name)

# get model output s3 from training job
huggingface_estimator_loaded.model_data